CNN

We will do convolution, pooling, convolution, pooling fully connected

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard
import time

In [2]:
dense_layer=[0]
layer_sizes=[68]
conv_layers=[3]

In [3]:
gpu_options=tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=.333)
sess=tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options))

In [4]:
import pickle
X=pickle.load(open('X.pickle','rb'))
y=pickle.load(open('y.pickle','rb'))

In [5]:
import numpy as np


In [6]:
y=np.array(y)#both inputs to the model have to be numpy array and not lists

we have to normalize the value ..since for pixel data we know that min is 0 and max is 255.. we can just divide by 255



In [7]:
X=X/255.0


In [ ]:
%load_ext tensorboard
%tensorboard --logdir=logs/


<IPython.core.display.Javascript object>

In [ ]:
import tensorboard
tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))

NameError: ignored

In [8]:
for d_layer in dense_layer:
  for l_size in layer_sizes:
    for c_layer in conv_layers:
      NAME="{}-conv-{}-nodes-{}-dense-{}".format(c_layer,l_size,d_layer,int(time.time()))
      tensorboard=TensorBoard(log_dir='logs/{}'.format(NAME))
      print(NAME)
      
      model=Sequential()#2*64 convolution layer
      #layer1
      model.add(Conv2D(l_size,(3,3),input_shape=X.shape[1:]))#after conv we can have either an activation or pooling-units, window size, ip shape
      model.add(Activation("relu"))
      model.add(MaxPooling2D(pool_size=(2,2)))
      for l in range(c_layer-1):
        model.add(Conv2D(l_size,(3,3)))
        model.add(Activation("relu"))
        model.add(MaxPooling2D(pool_size=(2,2)))

      
      #layer3
      model.add(Flatten())
      for d in range(d_layer):
        model.add(Dense(l_size))
        model.add(Activation("relu"))

      model.add(Dense(64))
      model.add(Activation("relu"))

      #op layer
      model.add(Dense(1))
      model.add(Activation("sigmoid"))

      model.compile(loss="binary_crossentropy",optimizer="adam",metrics=['accuracy'])

      model.fit(X,y, batch_size=32,epochs=10,validation_split=.1, callbacks=[tensorboard])
model.save('catvsdog-cnn.model')

3-conv-68-nodes-0-dense-1650797536
Epoch 1/10
655/655 [==============================] - 129s 196ms/step - loss: 0.6360 - accuracy: 0.6260 - val_loss: 0.5638 - val_accuracy: 0.7083
Epoch 2/10
655/655 [==============================] - 123s 187ms/step - loss: 0.5386 - accuracy: 0.7326 - val_loss: 0.5235 - val_accuracy: 0.7363
Epoch 3/10
655/655 [==============================] - 124s 189ms/step - loss: 0.4890 - accuracy: 0.7638 - val_loss: 0.4805 - val_accuracy: 0.7672
Epoch 4/10
655/655 [==============================] - 128s 196ms/step - loss: 0.4497 - accuracy: 0.7891 - val_loss: 0.4387 - val_accuracy: 0.8003
Epoch 5/10
655/655 [==============================] - 130s 199ms/step - loss: 0.4078 - accuracy: 0.8124 - val_loss: 0.4261 - val_accuracy: 0.8071
Epoch 6/10
655/655 [==============================] - 129s 197ms/step - loss: 0.3793 - accuracy: 0.8285 - val_loss: 0.3878 - val_accuracy: 0.8230
Epoch 7/10
655/655 [==============================] - 136s 207ms/step - loss: 0.3441 - ac

In [19]:
import tensorflow as tf
import cv2

categories=["dog","cat"]

def prepare(file_path):
  IMG_SIZE=50
  img_array=cv2.imread(file_path,cv2.IMREAD_GRAYSCALE)
  new_array=cv2.resize(img_array,(IMG_SIZE,IMG_SIZE))
  
  return new_array.reshape(-1,IMG_SIZE,IMG_SIZE)

model=tf.keras.models.load_model("catvsdog-cnn.model")


In [20]:
pre_image=prepare('/content/photo-1615789591457-74a63395c990.jpeg')


In [21]:
prediction=model.predict([pre_image])
print(pre_image)

[[[  5   4  14 ...  27  26  24]
  [  4   3  12 ...  33  28  23]
  [  1   2  11 ...  32  28  26]
  ...
  [105 110 110 ...  77  73  68]
  [108 109 106 ...  78  69  65]
  [100  96 102 ...  79  71  64]]]


In [22]:
print(prediction)
print(categories[int(prediction[0][0])])

[[1.]]
cat
